In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense

data = pd.read_csv('american_bankruptcy.csv')
label_encoder = LabelEncoder()
data['status_label'] = label_encoder.fit_transform(data['status_label'])

data = data.sort_values(by=['company_name', 'year'])

X = data.iloc[:, 3:].values
y = data['status_label'].values

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

n_timesteps = 2  
X_seq, y_seq = [], []

for i in range(n_timesteps, len(X)):
    X_seq.append(X[i - n_timesteps:i])
    y_seq.append(y[i])

X_seq = np.array(X_seq)
y_seq = np.array(y_seq)
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

rnn_model = Sequential([
    SimpleRNN(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(y_train.shape[1], activation='softmax')
])

rnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
rnn_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)


2024-11-02 14:53:12.213460: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/50
1771/1771 [==============================] - 5s 2ms/step - loss: 0.2504 - accuracy: 0.9319 - val_loss: 0.2466 - val_accuracy: 0.9330
Epoch 2/50
1771/1771 [==============================] - 3s 2ms/step - loss: 0.2429 - accuracy: 0.9341 - val_loss: 0.2447 - val_accuracy: 0.9330
Epoch 3/50
1771/1771 [==============================] - 3s 2ms/step - loss: 0.2425 - accuracy: 0.9341 - val_loss: 0.2447 - val_accuracy: 0.9330
Epoch 4/50
1771/1771 [==============================] - 4s 2ms/step - loss: 0.2424 - accuracy: 0.9341 - val_loss: 0.2444 - val_accuracy: 0.9330
Epoch 5/50
1771/1771 [==============================] - 4s 2ms/step - loss: 0.2425 - accuracy: 0.9341 - val_loss: 0.2442 - val_accuracy: 0.9330
Epoch 6/50
1771/1771 [==============================] - 3s 2ms/step - loss: 0.2422 - accuracy: 0.9341 - val_loss: 0.2456 - val_accuracy: 0.9330
Epoch 7/50
1771/1771 [==============================] - 4s 2ms/step - loss: 0.2422 - accuracy: 0.9341 - val_loss: 0.2439 - val_accuracy:

In [2]:
lstm_model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(y_train.shape[1], activation='softmax')  # Use sigmoid for binary classification
])

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)


Epoch 1/50
1771/1771 [==============================] - 5s 2ms/step - loss: 0.2537 - accuracy: 0.9341 - val_loss: 0.2458 - val_accuracy: 0.9330
Epoch 2/50
1771/1771 [==============================] - 3s 2ms/step - loss: 0.2429 - accuracy: 0.9341 - val_loss: 0.2448 - val_accuracy: 0.9330
Epoch 3/50
1771/1771 [==============================] - 4s 2ms/step - loss: 0.2427 - accuracy: 0.9341 - val_loss: 0.2447 - val_accuracy: 0.9330
Epoch 4/50
1771/1771 [==============================] - 4s 2ms/step - loss: 0.2426 - accuracy: 0.9341 - val_loss: 0.2446 - val_accuracy: 0.9330
Epoch 5/50
1771/1771 [==============================] - 4s 2ms/step - loss: 0.2425 - accuracy: 0.9341 - val_loss: 0.2457 - val_accuracy: 0.9330
Epoch 6/50
1771/1771 [==============================] - 4s 2ms/step - loss: 0.2424 - accuracy: 0.9341 - val_loss: 0.2445 - val_accuracy: 0.9330
Epoch 7/50
1771/1771 [==============================] - 4s 2ms/step - loss: 0.2422 - accuracy: 0.9341 - val_loss: 0.2446 - val_accuracy:

In [3]:
rnn_eval = rnn_model.evaluate(X_test, y_test, verbose=0)
print(f'RNN Test Loss: {rnn_eval[0]}, Test Accuracy: {rnn_eval[1]}')
lstm_eval = lstm_model.evaluate(X_test, y_test, verbose=0)
print(f'LSTM Test Loss: {lstm_eval[0]}, Test Accuracy: {lstm_eval[1]}')

RNN Test Loss: 0.24432553350925446, Test Accuracy: 0.9321936964988708
LSTM Test Loss: 0.2428002655506134, Test Accuracy: 0.9320666193962097
